# openelec example walkthrough
This notebook goes through the main functionality of the openelec model.
There is more lurking in the code but this should get you started.

In [ ]:
from pathlib import Path
import os

import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import folium

from openelec.local import LocalModel
from openelec.national import NationalModel
from openelec import conv

In [ ]:
# Hide Fiona warnings
import warnings
warnings.simplefilter("ignore")

### Set input and output folders

In [ ]:
test_data = Path('test_data')
test_output = Path('test_output')

# National
Run openelec at the national level. This creates a plan of which technology to use to connect each cluster in the country, and creates a plan for network extensions and densification, as well as where off-grid technology should be used.

In [ ]:
clusters_in = test_data / 'clusters.geojson'
national_out = test_output / 'national'

### These are the primary model parameters.

In [ ]:
nm = NationalModel(data=clusters_in)
nm.parameters(actual_pop=2.2e6,      # current population (NOT USED)
              pop_growth=0.01,       # %
              gdp_growth=0.02,       # %
              people_per_hh=5,       # 
              
              grid_mv_cost=50,       # USD/m
              grid_lv_cost=3,        # USD/m
              grid_trans_cost=3500,  # USD/unit
              grid_conn_cost=200,    # USD/hh
              grid_opex_ratio=0.02,  # % of capex (NOT USED)
              
              mg_gen_cost=4000,      # USD/kW
              mg_lv_cost=2,          # USD/m
              mg_conn_cost=100,      # USD/hh
              mg_opex_ratio=0.02,    # % of capex (NOT USED)
              
              access_tot=0.30,       # total access rate
              access_urban=0.66,     # urban access rate
              
              grid_dist_connected=2, # km, less considered connected
              minimum_pop=100,       # exclude any population below this
              demand_factor=5,       # Demand = demand_factor * log(gdp)
              use_mtf=False)         # whether to overide demand formula and use MTF

## Run dynamically with 4 steps of 5 years each
In this mode, the model is run repeatedly, each time getting closer to universal electrification.
Each step only includes the most profitable proportion of new on/off-grid connections and densification.

In [ ]:
steps = 4                             # number of steps to use (note the graph will break if not == 4)
years_per_step = 5                    # the number of years between steps

In [ ]:
dynamic_model = nm.dynamic(steps=steps, years_per_step=years_per_step)
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12,12))
features = {}
results = {}
years = [2025, 2030, 2035, 3040]

for i, ((t, n, r), ax) in enumerate(zip(dynamic_model, (ax1, ax2, ax3, ax4))):
    features[f't{i+1}'] = t
    features[f'n{i+1}'] = n
    results[i+1] = r
    
    n.plot(ax=ax, column='existing', cmap='Set1')
    t.plot(ax=ax, column='type', cmap='tab10')
    ax.text(nm.x_mean, nm.y_mean, years[i], dict(size=30))
    ax.axis('off')

conv.save_to_path(national_out, **features)
plt.subplots_adjust(hspace=-0.3, wspace=0.0)
plt.show()
print('Done dynamic model and saved to', national_out.absolute())

These columns show the results for each step.

In [ ]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)
summary = pd.DataFrame(results)
summary.columns = [f'Step {c}' for c in summary.columns]
summary

# Run a single simulation for the whole time period
In this mode, the model is only run once, with a target of 100% electrification at an undefined future point.

In [ ]:
nm.setup(sort_by='pop')
nm.initial_access()
nm.demand_levels()
nm.connect_targets()
nm.model()
nm.spatialise()
results = nm.summary()
nm.save_to_path(national_out)
print('Done single model and saved to', national_out.absolute())

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))  
nm.network_out.plot(ax=ax, column='existing', cmap='Set1')
nm.targets_out.plot(ax=ax, column='type', cmap='tab10')
ax.axis('off')
plt.show()

In [ ]:
print('\t\t\t')
print(f'New grid: {results["new-grid"]} at cost ${results["cost-grid"]:,.0f}')
print(f'New off-grid: {results["new-off-grid"]} at cost ${results["cost-off-grid"]:,.0f}')
print(f'Densify: {results["densify"]} at cost ${results["cost-densify"]:,.0f}')
print()
print(f'Total cost ${results["tot-cost"]:,.0f}')
print()
print(f'Modelled pop: {results["model-pop"]:,.0f}')
print(f'Currently electrified pop: {results["already-elec-pop"]:,.0f}')
print(f'Densify pop: {results["densify-pop"]:,.0f}')
print(f'New extentions pop: {results["new-conn-pop"]:,.0f}')
print(f'Off-grid pop: {results["new-og-pop"]:,.0f}')

# Local
This mode zooms in a particular village and calculates the optimum mini-grid and solar-home system set up to connect that village most profitably.

In [ ]:
buildings_in = test_data / 'buildings.geojson'
local_out = test_output / 'local'

In [ ]:
lm = LocalModel(data=buildings_in)
lm.setup(sort_by='area',
         min_area=0)  # m2, exclude buildings with area below this

In [ ]:
m = folium.Map([lm.y_mean, lm.x_mean],
               zoom_start=15,
               control_scale=True)

popup_html = '<p>Latitude: " + lat + "</p><p>Longitude: " + lng + "</p>'
folium.ClickForMarker(popup=popup_html).add_to(m)
folium.GeoJson(lm.targets).add_to(m)
print('Click on the desired location for the PV point.')
m

In [ ]:
latitude = -29.5432
longitude = 28.1339

In [ ]:
lm.connect_targets(origin=(latitude, longitude))

In [ ]:
lm.parameters(demand=8,             # average demand in kWh/person/month
              tariff=0.2,          # USD/kWh charged
              gen_cost=4000,        # USD/kW for installation, excluding distribution network
              cost_wire=20,         # USD per metre
              cost_connection=140,  # USD per connection/node
              opex_ratio=0.02,      # % of capex per year (values above 1 are divided by 100)
              years=20,             # years over which to amortize (and maintain)
              discount_rate=0.06)   # discount rate (values above 1 are divided by 100)

In [ ]:
lm.model()

In [ ]:
lm.spatialise()

fig, ax = plt.subplots(figsize=(10,10))
lm.network_out.loc[lm.network_out['enabled'] == 1].plot(ax=ax, color='red')
lm.targets_out.plot(ax=ax, color='blue')
ax.scatter(longitude, latitude, s=200, lw=0, c='green')
plt.show()

In [ ]:
lm.save_to_path(local_out)
print('Done local model and saved to', local_out.absolute())

In [ ]:
results = lm.summary()
print(f'Total houses connected: {results["connected"]} out of {len(lm.targets_out)-1}')
print(f'Generator installation size: {results["gen-size"]:.0f} kW')
print(f'Total length of lines is {results["line-length"]:.0f}m')
print(f'CAPEX: ${results["capex"]:.0f}')
print(f'Annual OPEX: ${results["opex"]:.0f}')
print(f'Annual Income: ${results["income"]:.0f}')
print(f'NPV over {lm.years} years is ${lm.results["npv"]:.0f}')